In [3]:
import nltk
import re
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/noahcowan-
[nltk_data]     maccallum/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
corpus = "Learning% makes 'me' happy. I am happy be-cause I am learning! :)"

# lowercase
corpus = corpus.lower()

# remove special characters
corpus = re.sub(r"[^a-zA-Z0-9.?! ]+", "", corpus)
# Note that this gets rid of emoticon, which is important for sentiment but not this task.

print(corpus)

learning makes me happy. i am happy because i am learning! 


In [7]:
# Split newlines into list of sentences
input_date="Sat May  9 07:33:35 CEST 2020"

print(input_date.split(' '))
print(input_date.split(' ')[4].split(':'))

['Sat', 'May', '', '9', '07:33:35', 'CEST', '2020']
['07', '33', '35']


In [8]:
# Split each sentence into a list of words, using nltk
sentence = 'i am happy because I am learning.'
tokenized_sentence = nltk.word_tokenize(sentence)
print(tokenized_sentence)

['i', 'am', 'happy', 'because', 'I', 'am', 'learning', '.']


In [9]:
# Find the length of each word
print([(word, len(word)) for word in tokenized_sentence])

[('i', 1), ('am', 2), ('happy', 5), ('because', 7), ('I', 1), ('am', 2), ('learning', 8), ('.', 1)]


In [10]:
# Sentence to n-gram. Window scans the list of words frmo beginning to end.corpus

def sentence_to_trigram(tokenized_sentence):
    # Prints all trigrams in the given tokenized sentence.
    for i in range(len(tokenized_sentence)):
        trigram = tokenized_sentence[i:i + 3]
        print(trigram)

sentence_to_trigram(tokenized_sentence)

['i', 'am', 'happy']
['am', 'happy', 'because']
['happy', 'because', 'I']
['because', 'I', 'am']
['I', 'am', 'learning']
['am', 'learning', '.']
['learning', '.']
['.']


In [11]:
# Prefix of an n-gram. This is needed to calculate the probability of an n-gram
# Get an n-1-gram prefix from n-gram

fourgram = ['i', 'am', 'happy', 'because']
trigram = fourgram[0:-1]
print(trigram)

['i', 'am', 'happy']


In [13]:
# Start and end of sentence word. Prepend n-1 to beginning and one to the end
n = 3
print(['<s>'] * (n-1) + tokenized_sentence + ['</s>'])

['<s>', '<s>', 'i', 'am', 'happy', 'because', 'I', 'am', 'learning', '.', '</s>']


In [2]:
# Building the language model
# count matrix - frequencies of n-grams and n-gram prefixes in the training dataset. Count of (n-1)-gram prefix followed by all possible last words in the vocabulary.

n_gram_counts = {
    ('i', 'am', 'happy'): 2,
    ('am', 'happy', 'because'): 1}

# get count for an n-gram tuple
print(n_gram_counts[('i', 'am', 'happy')])

# check if n-gram is present in the dict
if ('i', 'am', 'learning') in n_gram_counts:
    print('hit')
else:
    print('miss')

# concat tuple for prefix and tuple with the last word to create the n-gram
prefix = ('i', 'am', 'happy')
word = 'because'
n_gram = prefix + (word,)
print(n_gram)

2
miss
('i', 'am', 'happy', 'because')


In [5]:
# sliding window to populate trigram count matrix

import numpy as np
import pandas as pd
from collections import defaultdict

def single_pass_trigram_count_matrix(corpus):
    # corpus is pre-processed and tokenized
    # Returns list of all bigrams, vocabulary of all found words, dataframe with bigram prefixes as rows and vocab words as columns
    bigrams = []
    vocabulary = []
    count_matrix_dict = defaultdict(dict)

    # go through the corpus once with a sliding window
    for i in range(len(corpus) - 3 + 1):
        # The sliding window starts at position i and contains 3 words
        trigram = tuple(corpus[i:i+3])
        bigram = trigram[0:-1]
        if not bigram in bigrams:
            bigrams.append(bigram)

        last_word = trigram[-1]
        if not last_word in vocabulary:
            vocabulary.append(last_word)

        if (bigram, last_word) not in count_matrix_dict:
            count_matrix_dict[bigram, last_word] = 0
        
        count_matrix_dict[bigram,last_word] += 1
    
    print(count_matrix_dict)
    # convert the count_matrix to np.array to fill in the blanks
    count_matrix = np.zeros((len(bigrams), len(vocabulary)))
    for trigram_key, trigram_count in count_matrix_dict.items():
        count_matrix[bigrams.index(trigram_key[0]), \
                        vocabulary.index(trigram_key[1])] = \
                        trigram_count
    # np.array to pandas ataframe conversion
    count_matrix = pd.DataFrame(count_matrix, index=bigrams, columns=vocabulary)
    return bigrams, vocabulary, count_matrix

corpus = ['i', 'am', 'happy', 'because', 'i', 'am', 'learning', '.']

bigrams, vocabulary, count_matrix = single_pass_trigram_count_matrix(corpus)

print(count_matrix)

defaultdict(<class 'dict'>, {(('i', 'am'), 'happy'): 1, (('am', 'happy'), 'because'): 1, (('happy', 'because'), 'i'): 1, (('because', 'i'), 'am'): 1, (('i', 'am'), 'learning'): 1, (('am', 'learning'), '.'): 1})
                  happy  because    i   am  learning    .
(i, am)             1.0      0.0  0.0  0.0       1.0  0.0
(am, happy)         0.0      1.0  0.0  0.0       0.0  0.0
(happy, because)    0.0      0.0  1.0  0.0       0.0  0.0
(because, i)        0.0      0.0  0.0  1.0       0.0  0.0
(am, learning)      0.0      0.0  0.0  0.0       0.0  1.0


In [6]:
# Probability matrix
row_sums = count_matrix.sum(axis=1)
prob_matrix = count_matrix.div(row_sums, axis=0)

print(prob_matrix)

                  happy  because    i   am  learning    .
(i, am)             0.5      0.0  0.0  0.0       0.5  0.0
(am, happy)         0.0      1.0  0.0  0.0       0.0  0.0
(happy, because)    0.0      0.0  1.0  0.0       0.0  0.0
(because, i)        0.0      0.0  0.0  1.0       0.0  0.0
(am, learning)      0.0      0.0  0.0  0.0       0.0  1.0


In [10]:
# Find the probability of a trigram in the probability matrix
trigram = ('i', 'am', 'happy')
bigram = trigram[:-1]
print(bigram)

word = trigram[-1]
print(word)

trigram_probability = prob_matrix[word][bigram]
print(trigram_probability)

('i', 'am')
happy
0.5


In [13]:
# list all words in vocabulary starting with a given prefix
vocabulary = ['i', 'am', 'happy', 'because', 'learning', '.', 'have', 'you', 'seen','it', '?']
starts_with = 'ha'

print(f'words in vocabulary starting with prefix: {starts_with}')
for word in vocabulary:
    if word.startswith(starts_with):
        print(word)

words in vocabulary starting with prefix: ha
happy
have


In [15]:
# Language model evaluation
# Can randomly sample corpus to define test and validation set
import random

def train_validation_test_split(data, train_percent, validation_percent):
    random.seed(87)

    random.shuffle(data)

    train_size = int(len(data) * train_percent / 100)
    train_data = data[0:train_size]

    validation_size = int(len(data) * validation_percent / 100)
    validation_data = data[train_size:train_size + validation_size]

    test_data = data[train_size + validation_size:]

    return train_data, validation_data, test_data

data = [x for x in range(0,100)]
print(train_validation_test_split(data, 80, 10))


([28, 76, 5, 0, 62, 29, 54, 95, 88, 58, 4, 22, 92, 14, 50, 77, 47, 33, 75, 68, 56, 74, 43, 80, 83, 84, 73, 93, 66, 87, 9, 91, 64, 79, 20, 51, 17, 27, 12, 31, 67, 81, 7, 34, 45, 72, 38, 30, 16, 60, 40, 86, 48, 21, 70, 59, 6, 19, 2, 99, 37, 36, 52, 61, 97, 44, 26, 57, 89, 55, 53, 85, 3, 39, 10, 71, 23, 32, 25, 8], [78, 65, 63, 11, 49, 98, 1, 46, 15, 41], [90, 96, 82, 42, 35, 13, 69, 24, 94, 18])


In [17]:
# Perplexity
p = 10 ** (-250)
M = 100
print(p, M)
perplexity = p ** (-1/M)
print(perplexity)

1e-250 100
316.22776601683796


In [19]:
# Out of vocabulary words (OOV)

# Build the vocabulary form M most frequent words
from collections import Counter

M = 3

# Can use counter to build this
word_counts = {'happy': 5, 'because': 3, 'i': 2, 'am': 2, 'learning': 3, '.': 1}
vocabulary = Counter(word_counts).most_common(M)
print(vocabulary)

vocabulary = [w[0] for w in vocabulary]
print(vocabulary)

[('happy', 5), ('because', 3), ('learning', 3)]
['happy', 'because', 'learning']


In [20]:
# replace OOV words with <unk>
sentence = ['am', 'i', 'learning']
output_sentence = []
print(sentence)

for w in sentence:
    # test if word w is in vocabulary
    if w in vocabulary:
        output_sentence.append(w)
    else:
        output_sentence.append('<UNK>')

print(output_sentence)

['am', 'i', 'learning']
['<UNK>', '<UNK>', 'learning']


In [21]:
f = 3
word_counts = {'happy': 5, 'because': 3, 'i': 2, 'am': 2, 'learning':3, '.': 1}

for word, freq in word_counts.items():
    if freq == f:
        print(word)

because
learning


In [22]:
# If there are many <unk>, may get low perplexity, as shown
training_set = ['i', 'am', 'happy', 'because','i', 'am', 'learning', '.']
training_set_unk = ['i', 'am', '<UNK>', '<UNK>','i', 'am', '<UNK>', '<UNK>']

test_set = ['i', 'am', 'learning']
test_set_unk = ['i', 'am', '<UNK>']

M = len(test_set)
probability = 1
probability_unk = 1

# pre-calculated probabilities
bigram_probabilities = {('i', 'am'): 1.0, ('am', 'happy'): 0.5, ('happy', 'because'): 1.0, ('because', 'i'): 1.0, ('am', 'learning'): 0.5, ('learning', '.'): 1.0}
bigram_probabilities_unk = {('i', 'am'): 1.0, ('am', '<UNK>'): 1.0, ('<UNK>', '<UNK>'): 0.5, ('<UNK>', 'i'): 0.25}

# Go through test set and calculate its bigram probability
for i in range(len(test_set) - 2 + 1):
    bigram = tuple(test_set[i:i+2])
    probability = probability * bigram_probabilities[bigram]

    bigram_unk = tuple(test_set_unk[i:i+2])
    probability_unk = probability_unk * bigram_probabilities_unk[bigram_unk]

perplexity = probability ** (-1/M)
perplexity_unk = probability_unk ** (-1/M)

print(perplexity)
print(perplexity_unk)



1.2599210498948732
1.0


In [25]:
# Smoothing. Bad that unknown phrase gets same probability as one that is in the test set
def add_k_smoothing_probability(k, vocabulary_size, n_gram_count, n_gram_prefix_count):
    numerator = n_gram_count + k
    denominator = n_gram_prefix_count + k * vocabulary_size
    return numerator/denominator

trigram_probabilities = {('i', 'am', 'happy'): 2}
bigram_probabilities = {('am', 'happy'): 10}
vocabulary_size = 5
k = 1

probability_known_trigram = add_k_smoothing_probability(k, vocabulary_size, trigram_probabilities[('i', 'am', 'happy')], 
                           bigram_probabilities[( 'am', 'happy')])

probability_unknown_trigram = add_k_smoothing_probability(k, vocabulary_size, 0, 0)

print(probability_known_trigram)
print(probability_unknown_trigram)

0.2
0.2


In [26]:
# Back-off. Use lower order n-grams in case information about high order n-grams is missing
# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('i', 'am', 'happy'): 0}
bigram_probabilities = {( 'am', 'happy'): 0.3}
unigram_probabilities = {'happy': 0.4}

# this is the input trigram we need to estimate
trigram = ('are', 'you', 'happy')

# find the last bigram and unigram of the input
bigram = trigram[1: 3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# 0.4 is used as an example, experimentally found for web-scale corpuses when using the "stupid" back-off
lambda_factor = 0.4
probability_hat_trigram = 0

# search for first non-zero probability starting with trigram
# to generalize this for any order of n-gram hierarchy, 
# you could loop through the probability dictionaries instead of if/else cascade
if trigram not in trigram_probabilities or trigram_probabilities[trigram] == 0:
    print(f"probability for trigram {trigram} not found")
    
    if bigram not in bigram_probabilities or bigram_probabilities[bigram] == 0:
        print(f"probability for bigram {bigram} not found")
        
        if unigram in unigram_probabilities:
            print(f"probability for unigram {unigram} found\n")
            probability_hat_trigram = lambda_factor * lambda_factor * unigram_probabilities[unigram]
        else:
            probability_hat_trigram = 0
    else:
        probability_hat_trigram = lambda_factor * bigram_probabilities[bigram]
else:
    probability_hat_trigram = trigram_probabilities[trigram]

print(f"probability for trigram {trigram} estimated as {probability_hat_trigram}")



besides the trigram ('are', 'you', 'happy') we also use bigram ('you', 'happy') and unigram (happy)

probability for trigram ('are', 'you', 'happy') not found
probability for bigram ('you', 'happy') not found
probability for unigram happy found

probability for trigram ('are', 'you', 'happy') estimated as 0.06400000000000002


In [27]:
# Interpolation. use weighted probabilities of  n-grams of all orders every time, not just when high order information is missing

# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('i', 'am', 'happy'): 0.15}
bigram_probabilities = {( 'am', 'happy'): 0.3}
unigram_probabilities = {'happy': 0.4}

# the weights come from optimization on a validation set
lambda_1 = 0.8
lambda_2 = 0.15
lambda_3 = 0.05

# this is the input trigram we need to estimate
trigram = ('i', 'am', 'happy')

# find the last bigram and unigram of the input
bigram = trigram[1: 3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# in the production code, you would need to check if the probability n-gram dictionary contains the n-gram
probability_hat_trigram = lambda_1 * trigram_probabilities[trigram] 
+ lambda_2 * bigram_probabilities[bigram]
+ lambda_3 * unigram_probabilities[unigram]

print(f"estimated probability of the input trigram {trigram} is {probability_hat_trigram}")


besides the trigram ('i', 'am', 'happy') we also use bigram ('am', 'happy') and unigram (happy)

estimated probability of the input trigram ('i', 'am', 'happy') is 0.12
